In [1]:
!pip install spektral -qq
!pip install --upgrade keras -qq
!pip install ogb -qq
!git clone https://github.com/anas-rz/k3-node.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Cloning into 'k3-node'...
remote: Enumerating objects: 484, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 484 (delta 38), reused 63 (delta 26), pack-reused 401
Receiving objects: 100% (484/484), 344.27 KiB | 26.48 MiB/s, done.
Resolving deltas: 100% (274/274), done.


In [2]:
import os, sys
os.environ['KERAS_BACKEND'] = 'tensorflow'
sys.path.append('/content/k3-node')

In [18]:
import numpy as np
from ogb.nodeproppred import NodePropPredDataset
from keras.layers import BatchNormalization, Dropout, Input
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy
from keras.models import Model
from keras.optimizers import Adam

from spektral.datasets.ogb import OGB
from spektral.transforms import AdjToSpTensor, GCNFilter

from pprint import pprint

from k3_node.layers import ARMAConv

In [10]:
# Load data
dataset_name = "ogbn-arxiv"
ogb_dataset = NodePropPredDataset(dataset_name)
dataset = OGB(ogb_dataset, transforms=[GCNFilter(), AdjToSpTensor()])
graph = dataset[0]
x, adj, y = graph.x, graph.a, graph.y

In [20]:
# Parameters
channels = 256  # Number of channels for GCN layers
dropout = 0.5  # Dropout rate for the features
learning_rate = 1e-2  # Learning rate
epochs = 200  # Number of training epochs
N = dataset.n_nodes  # Number of nodes in the graph
F = dataset.n_node_features  # Original size of node features
n_out = ogb_dataset.num_classes  # OGB labels are sparse indices

In [21]:
# Data splits
idx = ogb_dataset.get_idx_split()
idx_tr, idx_va, idx_te = idx["train"], idx["valid"], idx["test"]
mask_tr = np.zeros(N, dtype=bool)
mask_va = np.zeros(N, dtype=bool)
mask_te = np.zeros(N, dtype=bool)
mask_tr[idx_tr] = True
mask_va[idx_va] = True
mask_te[idx_te] = True
masks = [mask_tr, mask_va, mask_te]

In [22]:
# Model definition
x_in = Input(shape=(F,))
a_in = Input((N,), sparse=True)
x_1 = ARMAConv(channels, activation="relu")([x_in, a_in])
x_1 = BatchNormalization()(x_1)
x_1 = Dropout(dropout)(x_1)
x_2 = ARMAConv(channels, activation="relu")([x_1, a_in])
x_2 = BatchNormalization()(x_2)
x_2 = Dropout(dropout)(x_2)
x_3 = ARMAConv(n_out, activation="softmax")([x_2, a_in])

In [23]:
# Build model
model = Model(inputs=[x_in, a_in], outputs=x_3)
optimizer = Adam(learning_rate=learning_rate)
loss_fn = SparseCategoricalCrossentropy()
acc_metric = SparseCategoricalAccuracy()
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃    Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 128)            │          0 │ -                          │
│ (InputLayer)              │                        │            │                            │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ input_layer_3             │ (None, 169343)         │          0 │ -                          │
│ (InputLayer)              │                        │            │                            │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ arma_conv_3 (ARMAConv)    │ (None, 256)            │     65,792 │ input_layer_2[0][0],       │
│                           │                        │            │ input_layer_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ batch_normalization_2     │ (None, 256)            │      1,024 │ arma_conv_3[0][0]          │
│ (BatchNormalization)      │                        │            │                            │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ dropout_6 (Dropout)       │ (None, 256)            │          0 │ batch_normalization_2[0][… │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ arma_conv_4 (ARMAConv)    │ (None, 256)            │    131,328 │ dropout_6[0][0],           │
│                           │                        │            │ input_layer_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ batch_normalization_3     │ (None, 256)            │      1,024 │ arma_conv_4[0][0]          │
│ (BatchNormalization)      │                        │            │                            │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ dropout_8 (Dropout)       │ (None, 256)            │          0 │ batch_normalization_3[0][… │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ arma_conv_5 (ARMAConv)    │ (None, 40)             │     20,520 │ dropout_8[0][0],           │
│                           │                        │            │ input_layer_3[0][0]        │
└───────────────────────────┴────────────────────────┴────────────┴────────────────────────────┘

 Total params: 219,688 (858.16 KB)

 Trainable params: 218,664 (854.16 KB)

 Non-trainable params: 1,024 (4.00 KB)

In [24]:
import tensorflow as tf
# Training function
@tf.function
def train(inputs, target, mask):
    acc_metric.reset_state()
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target[mask], predictions[mask]) + sum(model.losses)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    acc_metric.update_state(target[mask], predictions[mask])
    return loss, acc_metric.result()

In [25]:
@tf.function
def evaluate(inputs, target, mask):
    acc_metric.reset_state()
    predictions = model(inputs, training=True)
    loss = loss_fn(target[mask], predictions[mask]) + sum(model.losses)
    acc_metric.update_state(target[mask], predictions[mask])
    return loss, acc_metric.result()

In [26]:
# Train model
for i in range(1, 1 + epochs):
    tr_loss, tr_acc = train([x, adj], y, mask_tr)
    eval_loss, eval_acc = evaluate([x, adj], y, mask_va) # TODO Add more metrics
    pprint(f"EPOCH {i}: Training Loss {tr_loss.numpy()} - Training Accuracy {tr_acc}, Validation Loss: {eval_loss} - Validation Accuracy {eval_acc}")
test_loss, test_acc = evaluate([x, adj], y, mask_te)
pprint(f"Test Loss: {test_loss} Test Accuracy: {test_acc}")

('EPOCH 1: Training Loss 6.727264881134033 - Training Accuracy '
 '0.016780110076069832, Validation Loss: 3.410824775695801 - Validation '
 'Accuracy 0.3783012926578522')
('EPOCH 2: Training Loss 3.9297218322753906 - Training Accuracy '
 '0.3478848934173584, Validation Loss: 2.8849685192108154 - Validation '
 'Accuracy 0.4568609595298767')
('EPOCH 3: Training Loss 3.318859815597534 - Training Accuracy '
 '0.4080997705459595, Validation Loss: 2.4148788452148438 - Validation '
 'Accuracy 0.5107554197311401')
('EPOCH 4: Training Loss 2.793854236602783 - Training Accuracy '
 '0.4620468318462372, Validation Loss: 2.2219178676605225 - Validation '
 'Accuracy 0.521762490272522')
('EPOCH 5: Training Loss 2.5515968799591064 - Training Accuracy '
 '0.4713165760040283, Validation Loss: 2.018460512161255 - Validation Accuracy '
 '0.5272995829582214')
('EPOCH 6: Training Loss 2.343061685562134 - Training Accuracy '
 '0.4847428500652313, Validation Loss: 1.9317702054977417 - Validation '
 'Accuracy 